# Vault Management Notebook
Start by setting the Vault address:

In [ ]:
# Initialize the variable for the Vault address
vault_address = "https://192.168.1.1:8200"
print(f"Vault address is set to: {vault_address}")

## Unseal Vault
Proceed with unsealing the Vault using your unseal keys:

In [ ]:
# Define unseal keys
unseal_key1 = "FOOBAR1"
unseal_key2 = "FOOBAR2"
unseal_key3 = "FOOBAR3"

print(f"Vault unseal keys are configured as follows: \n {unseal_key1} \n {unseal_key2} \n {unseal_key3}")

In [ ]:
%%bash -s "$vault_address" "$unseal_key1" "$unseal_key2" "$unseal_key3"

# Assign command line arguments to variables
vault_address=$1
unseal_key1=$2
unseal_key2=$3
unseal_key3=$4

# Iterate through each key to unseal the Vault
declare -a keys=($unseal_key1 $unseal_key2 $unseal_key3)
for key in "${keys[@]}"; do
    response=$(
        curl -s -k -X POST \
            -H "Content-Type: application/json" \
            -d "{\"key\": \"$key\"}" \
            $vault_address/v1/sys/unseal
    )
    
    echo "$response"
    
    if [[ $(echo $response | grep -o '"sealed":false') ]]; then
       echo "Vault is unsealed."
       break
    fi
done

#### Testing Current Vault API Key
Navigate to `MicroK8s` > `ConfigMaps and Secrets` > `vault-api-key` in the default namespace to find the Vault API key:

In [ ]:
# Set Vault API key
vault_api_key = "hvs.FOOBAR"
print(f"Vault API key is set to: {vault_api_key}")

In [ ]:
%%bash -s "$vault_address" "$vault_api_key"

# Assign command line arguments to variables
vault_address=$1
vault_api_key=$2

# Test the validity of the Vault API token
response=$(
     curl -s -k -X POST \
          -H "X-Vault-Token: $vault_api_key" \
          -H "Content-Type: application/json" \
          -d '{"token": "username=kleberf,password=theUserPassword"}' \
          $vault_address/v1/users/243478253677838381/samba
)

echo "$response"

if [[ -z "$response" ]]; then
     echo "Token appears to be valid."
fi

### Token Update Warning
A `permission denied` error indicates the token is no longer valid.

<font color=#ff0000>**Proceed with caution:**</font> the following steps involve sensitive operations and should be handled securely. The root token is required for these operations and should never be exposed, shared, or stored insecurely, as it can decrypt all secrets.

In [ ]:
# Initialize the Vault root token
vault_root_token = "hvs.FOOBAR"
print(f"Vault root token is set to: {vault_root_token}")

In [ ]:
%%bash -s "$vault_address" "$vault_root_token"

vault_address=$1
vault_root_token=$2

# Create a new token with specific policies
response=$(
    curl -s -k -X POST \
        -H "X-Vault-Token: $vault_root_token" \
        -H "Content-Type: application/json" \
        -d '{"policies": ["user-samba-update"], "meta": {"description": "Token for Samba Password Updates"}, "no_parent": true, "ttl": "0s", "renewable": false}' \
        $vault_address/v1/auth/token/create
    )

echo "$response"